In [181]:
#libraries we need
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import date
pd.options.mode.chained_assignment = None
import h2o
from sklearn.model_selection import train_test_split
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.grid.grid_search import H2OGridSearch

In [195]:
h2o.init(max_mem_size='24G', nthreads=4) # start h2o with plenty of memory and threads
h2o.remove_all()                         # clears h2o memory
h2o.no_progress()                        # turn off h2o progress indicators

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,2 hours 59 mins
H2O_cluster_timezone:,America/Los_Angeles
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.5
H2O_cluster_version_age:,6 days
H2O_cluster_name:,H2O_from_python_vibhz_r14fx0
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,21.25 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [196]:
#Import cleaned files 
train_df = pd.read_csv('./House-Prices-Data/new_train.csv')
test_df = pd.read_csv('./House-Prices-Data/new_test.csv').drop(['SalePrice'],axis=1)
#train_df.head(5)

In [197]:
train_df = pd.get_dummies(train_df)
test_df = pd.get_dummies(test_df)

In [198]:
train_y = train_df['SalePrice']
train_x = train_df.drop('SalePrice', axis = 1)

In [199]:
response = 'SalePrice'
predictors = list(X_train.columns.values)

In [200]:
hf=h2o.H2OFrame(train_df)
gf=h2o.H2OFrame(test_df)

In [205]:
hyper_params = {'alpha': [0, .25, .5, .75, .1]
                ,'lambda':[1, 0.5, 0.1, 0.01, 0.001, 0.0001, 0.00001, 0]
               }

glm = H2OGeneralizedLinearEstimator(family = 'gaussian',standardize = True,lambda_search = True)

# build grid search with previously made GLM and hyperparameters
grid = H2OGridSearch(model = glm, hyper_params = hyper_params,
                     search_criteria = {'strategy': "Cartesian"})


grid.train(x = predictors, y = response, training_frame = hf,nfolds=5,seed=1)


In [206]:
sorted_grid = grid.get_grid(sort_by='RMSE', decreasing=False)
best_model = sorted_grid.models[0]
best_model.cross_validation_metrics_summary()


Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,mae,17607.871,1146.3462,19647.191,17004.912,16979.883,17284.154,17123.217
1,mean_residual_deviance,1.07257357E9,3.27831136E8,1.23665434E9,9.2511117E8,1.56560499E9,7.5342944E8,8.8206797E8
2,mse,1.07257357E9,3.27831136E8,1.23665434E9,9.2511117E8,1.56560499E9,7.5342944E8,8.8206797E8
3,null_deviance,1.84226152E12,2.70834991E11,2.2061714E12,1.58684388E12,1.77678608E12,1.60800283E12,2.0335037E12
4,r2,0.8279846,0.05642642,0.8334983,0.8326571,0.733001,0.86462283,0.8761436
5,residual_deviance,3.15109736E11,1.05163612E11,3.67286354E11,2.65506898E11,4.74378306E11,2.16987681E11,2.51389362E11
6,rmse,32459.553,4867.1147,35166.098,30415.64,39567.727,27448.67,29699.629
7,rmsle,0.14254805,0.009989087,0.15076812,0.1260156,0.14687236,0.148516,0.14056817


In [211]:
pred =  best_model.predict(gf)


/Users/vibhz/anaconda3/lib/python3.6/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset is missing column 'Utilities_NoSeWa': substituting in a column of NaN
  warnings.warn(w)
/Users/vibhz/anaconda3/lib/python3.6/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset is missing column 'Condition2_RRAe': substituting in a column of NaN
  warnings.warn(w)
/Users/vibhz/anaconda3/lib/python3.6/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset is missing column 'Condition2_RRAn': substituting in a column of NaN
  warnings.warn(w)
/Users/vibhz/anaconda3/lib/python3.6/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset is missing column 'Condition2_RRNn': substituting in a column of NaN
  warnings.warn(w)
/Users/vibhz/anaconda3/lib/python3.6/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset is missing column 'HouseStyle_2.5Fin': substituting in a column of NaN
  warnings.warn(w)
/Users/vibhz/anaconda3/lib/python3.6/site-

predict
110067
163826
179266
191838
212655
168863
176165
165639
182045
116920


In [212]:
pred

predict
110067
163826
179266
191838
212655
168863
176165
165639
182045
116920


In [208]:
print(grid.model_performance())

{'Grid_GLM_Key_Frame__upload_a9789357483bf475fa4bb65210064d3f.hex_model_python_1593042533260_28_model_10': None, 'Grid_GLM_Key_Frame__upload_a9789357483bf475fa4bb65210064d3f.hex_model_python_1593042533260_28_model_3': None, 'Grid_GLM_Key_Frame__upload_a9789357483bf475fa4bb65210064d3f.hex_model_python_1593042533260_28_model_6': None, 'Grid_GLM_Key_Frame__upload_a9789357483bf475fa4bb65210064d3f.hex_model_python_1593042533260_28_model_7': None, 'Grid_GLM_Key_Frame__upload_a9789357483bf475fa4bb65210064d3f.hex_model_python_1593042533260_28_model_4': None, 'Grid_GLM_Key_Frame__upload_a9789357483bf475fa4bb65210064d3f.hex_model_python_1593042533260_28_model_8': None, 'Grid_GLM_Key_Frame__upload_a9789357483bf475fa4bb65210064d3f.hex_model_python_1593042533260_28_model_2': None, 'Grid_GLM_Key_Frame__upload_a9789357483bf475fa4bb65210064d3f.hex_model_python_1593042533260_28_model_5': None, 'Grid_GLM_Key_Frame__upload_a9789357483bf475fa4bb65210064d3f.hex_model_python_1593042533260_28_model_1': None,

In [ ]:
best_model.train()